<a href="https://colab.research.google.com/github/Jess-cah/measure-pineapple/blob/main/detector/maskpine_inspect_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN")
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN'

In [ ]:
!pip install -r requirements.txt

In [ ]:
%tensorflow_version 1.x
!pip3 install Keras==2.0.8

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/samples/pineappleResNet50_aug")
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/samples/pineappleResNet50_aug'

In [ ]:
import tensorflow as tf
import matplotlib.patches as patches

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import pineapple

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


# Configurations

In [ ]:
config = pineapple.PineappleConfig()
PINEAPPLE_DIR = os.path.join(ROOT_DIR, "datasets/pineapple160")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

#Notebook preferences

In [ ]:
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

#Load validation set

In [ ]:
# Load validation dataset
dataset = pineapple.PineappleDataset()
dataset.load_pineapple(PINEAPPLE_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 32
Classes: ['BG', 'pineapple']


#Load model

In [ ]:
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
#USED THIS ONE 
weights_path = "/content/drive/MyDrive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/logs/pineapple20201116T1748/mask_rcnn_pineapple_0029.h5"

# Or, load the last model you trained
#weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)


Loading weights  /content/drive/MyDrive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/logs/pineapple20201116T1748/mask_rcnn_pineapple_0029.h5





Re-starting from epoch 29


# Evaluate model

In [ ]:
# USING APPROACH FROM TRAIN_SHAPES.IPYNB - USING COMPUTE_AP FUNCTION
# https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb
# Compute VOC-Style mAP @ IoU=0.5
inference_config = InferenceConfig()
image_ids = dataset.image_ids
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

mAP:  1.0


In [ ]:
# USING COMPUTE_AP_RANGE FROM UTILS
# using code from https://www.kaggle.com/ipythonx/keras-global-wheat-detection-with-mask-rcnn

# keep track of how long it takes
%%time

thresh_score = [0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8, 0.85, 0.9, 0.95]

def evaluate_threshold_range(test_set, image_ids, model, 
                             iou_thresholds, inference_config):
    '''Calculate mAP based on iou_threshold range
    inputs:
        test_set        : test samples
        image_ids       : image ids of the test samples
        model           : trained model
        inference_config: test configuration
        iou_threshold   : by default [0.5:0.95:0.05]
    return:
        AP : mAP[@0.5:0.95] scores lists of the test samples
    '''
    # placeholder for all the ap of all classes for IoU socres 0.5 to 0.95 with step size 0.05
    AP = []
    np.seterr(divide='ignore', invalid='ignore') 
    
    for image_id in image_ids:
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(test_set, inference_config,
                                   image_id, use_mini_mask=False)

        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        AP_range = utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask, 
                                          r["rois"], r["class_ids"], r["scores"], r['masks'],
                                          iou_thresholds=iou_thresholds, verbose=0)
        
        if math.isnan(AP_range):
            continue
            
        # append the scores of each samples
        AP.append(AP_range)   
        
    return AP

AP = evaluate_threshold_range(dataset, dataset.image_ids,
                              model, thresh_score, inference_config)

print("AP[0.5:0.95]: ", np.mean(AP))


AP[0.5:0.95]:  0.9026214366853127
CPU times: user 12min 5s, sys: 10.8 s, total: 12min 16s
Wall time: 6min 45s
